In [208]:
import numpy as np
import pandas as pd 
import matplotlib.pyplot as plt
import seaborn as sns

In [209]:
data=pd.read_csv('trainandtest.csv', delimiter=';')

In [210]:
#Add new column 'Title' that keeps only titles in 'Name' column
data["Name"]= data["Name"].str.split(".", n = 1, expand = True) 
new = data["Name"].str.split(",", n = 1, expand = True) 
data["First Name"]= new[0]
data["Title"]= new[1]
data.drop(columns =["Name"], inplace = True)
data.drop(columns=["First Name"], inplace= True)

In [211]:
#Another view on family 
def process_family():
   
    global data
    # introducing a new feature : the size of families (including the passenger)
    data['FamilySize'] = data['Parch'] + data['SibSp'] + 1
    
    # introducing other features based on the family size
    data['Singleton'] = data['FamilySize'].map(lambda s: 1 if s == 1 else 0)
    data['SmallFamily'] = data['FamilySize'].map(lambda s: 1 if 2 <= s <= 4 else 0)
    data['LargeFamily'] = data['FamilySize'].map(lambda s: 1 if 5 <= s else 0)
    
  
    return data

data = process_family()
data.head()

,PassengerId,Survived,Pclass,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked,Title,FamilySize,Singleton,SmallFamily,LargeFamily
0,1,0.0,3,male,22.0,1,0,A/5 21171,7.2500,NaN,S,Mr,2,0,1,0
1,2,1.0,1,female,38.0,1,0,PC 17599,71.2833,C85,C,Mrs,2,0,1,0
2,3,1.0,3,female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S,Miss,1,1,0,0
3,4,1.0,1,female,35.0,1,0,113803,53.1000,C123,S,Mrs,2,0,1,0
4,5,0.0,3,male,35.0,0,0,373450,8.0500,NaN,S,Mr,1,1,0,0


In [212]:
#'Sex'become a dummy variable
data["Sex"] = pd.get_dummies(data["Sex"])

In [213]:
#Round the age to the nearest integer:
data["Age"]=round(data["Age"])

In [214]:
#Missing value age with combination of 'Sex'and 'Pclass'column'
guess_ages = np.zeros((2,3))

for i in range(0, 2):
    for j in range(0, 3):
        guess_data = data[(data['Sex'] == i) & (data['Pclass'] == j+1)]['Age'].dropna()
        age_guess = guess_data.median()
        # Convert random age float to nearest .5 age
        guess_ages[i,j] = int( age_guess/0.5 + 0.5 ) * 0.5
 
def wrangle_age(dataset):
    for i in range(0, 2):
        for j in range(0, 3):
            dataset.loc[ (dataset.Age.isnull()) & (dataset.Sex == i) & (dataset.Pclass == j+1),'Age'] = guess_ages[i,j]
    dataset['Age'] = dataset['Age'].astype(int)
    return dataset
 
data = wrangle_age(data)

In [215]:
#Treat Cabin differently
#Create a Column Letter of the Cabin 
data['CabinLetter'] = data['Cabin'].astype(str).str[0]

#Create a Column first number of the Cabin 
data.loc[:, 'CabinNumber']=0

for i in range(data['CabinNumber'].shape[0]):
    data['CabinNumber'][i]=data.Cabin.str[1][i]


#Fill missing Values 
data.CabinNumber = data.CabinNumber.fillna('0')

#Fill missing Values 

#Class 1: A,B,C 
#Class 2: D
#Class 3: E,F,G
#Notice that Missing values are globally for class 3
#Fill MV of 'Deck'column 
class1=['A','B','C']
class2=['D']
class3=['E','F','G']

for i in range(data['CabinLetter'].shape[0]):
    if (data['CabinLetter'][i]=='n'):
        if (data['Pclass'][i]==1) :
            data['CabinLetter'][i]=np.random.choice(class1)
        elif (data['Pclass'][i]==2) :
            data['CabinLetter'][i]=np.random.choice(class2)
        else :
            data['CabinLetter'][i]=np.random.choice(class3)




/Users/jennifervial/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if __name__ == '__main__':
/Users/jennifervial/anaconda3/lib/python3.6/site-packages/pandas/core/indexing.py:194: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._setitem_with_indexer(indexer, value)
/Users/jennifervial/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:33: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/Users/jennifervial/anaconda3/

In [216]:
#Traiter la colonne Embarked
data[pd.isnull(data["Embarked"])]

(data["Embarked"]).value_counts("S")*100
# We notice that 70% of passengers embarked in Southampton, we're going to affect by Naive Method, S to the two missing values.


data.Embarked = data.Embarked.fillna("S")

In [217]:
#Traiter la colonne Fare

data[pd.isnull(data["Fare"])]

data['Fare'].iloc[1043]= data['Fare'][data['Pclass'] == 3].dropna().median()
print(data['Fare'].iloc[1043])

8.05


/Users/jennifervial/anaconda3/lib/python3.6/site-packages/pandas/core/indexing.py:194: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._setitem_with_indexer(indexer, value)


In [218]:
#réagencement des colonnes:
inserted_cols = ['PassengerId', 'Survived','Sex', 'Age', 'Title','Relatives','Pclass','CabinNumber','CabinLetter','Embarked','Fare']
cols = ([col for col in inserted_cols if col in data] 
        + [col for col in data if col not in inserted_cols])
data = data[cols]
data.head()

,PassengerId,Survived,Sex,Age,Title,Pclass,CabinNumber,CabinLetter,Embarked,Fare,SibSp,Parch,Ticket,Cabin,FamilySize,Singleton,SmallFamily,LargeFamily
0,1,0.0,0,22,Mr,3,0,F,S,7.2500,1,0,A/5 21171,NaN,2,0,1,0
1,2,1.0,1,38,Mrs,1,8,C,C,71.2833,1,0,PC 17599,C85,2,0,1,0
2,3,1.0,1,26,Miss,3,0,F,S,7.9250,0,0,STON/O2. 3101282,NaN,1,1,0,0
3,4,1.0,1,35,Mrs,1,1,C,S,53.1000,1,0,113803,C123,2,0,1,0
4,5,0.0,0,35,Mr,3,0,F,S,8.0500,0,0,373450,NaN,1,1,0,0


In [219]:
# Keep tickets and process it with dummies variables 
data['TicketLetter'] = data['Ticket'].apply(lambda x : str(x)[0])
    
    
data.groupby(['TicketLetter'])['Survived'].mean().sort_values(ascending=False)
sns.barplot(x = 'TicketLetter', y='Survived', data=data)
data_count = data.groupby(['TicketLetter'],as_index=True)['PassengerId'].count().sort_values(ascending=False)
print(data_count)

def filter_ticket(x):
    if x in ['9', '8', '5', 'L', '6', 'F', '7', '4', 'W', 'A']:
        return 'Rare'
    elif x in ['C', 'P', 'S', '1']:
        return 'Frequent'
    elif x == '2':
        return 'Common'
    elif x == '3':
        return 'Commonest'


data['TicketCategory'] = data['TicketLetter'].apply(filter_ticket)
    
def simplify_ticket(data):
    data = pd.get_dummies(data, columns=['TicketCategory'])
    data = data.drop(['TicketLetter', 'Ticket'], axis=1)
    return data

data = simplify_ticket(data)
data.head()

/Users/jennifervial/anaconda3/lib/python3.6/site-packages/seaborn/categorical.py:1460: FutureWarning: remove_na is deprecated and is a private function. Do not use.
  stat_data = remove_na(group_data)


TicketLetter
3    429
2    278
1    210
S     98
P     98
C     77
A     42
W     19
F     13
7     13
4     11
6      9
L      5
5      3
9      2
8      2
Name: PassengerId, dtype: int64


,PassengerId,Survived,Sex,Age,Title,Pclass,CabinNumber,CabinLetter,Embarked,Fare,...,Parch,Cabin,FamilySize,Singleton,SmallFamily,LargeFamily,TicketCategory_Common,TicketCategory_Commonest,TicketCategory_Frequent,TicketCategory_Rare
0,1,0.0,0,22,Mr,3,0,F,S,7.2500,...,0,NaN,2,0,1,0,0,0,0,1
1,2,1.0,1,38,Mrs,1,8,C,C,71.2833,...,0,C85,2,0,1,0,0,0,1,0
2,3,1.0,1,26,Miss,3,0,F,S,7.9250,...,0,NaN,1,1,0,0,0,0,1,0
3,4,1.0,1,35,Mrs,1,1,C,S,53.1000,...,0,C123,2,0,1,0,0,0,1,0
4,5,0.0,0,35,Mr,3,0,F,S,8.0500,...,0,NaN,1,1,0,0,0,1,0,0


In [220]:
data['CabinLetter'].unique()

array(['F', 'C', 'E', 'D', 'G', 'A', 'B', 'T'], dtype=object)

In [221]:
data['Title'] = data['Title'].map({' Mr': 1, ' Mrs': 2, ' Miss': 3, ' Master': 4, ' Don': 5, ' Rev': 6, ' Dr': 7,' Mme':8,' Ms':9,' Major':10,' Lady':11,' Sir':12,' Mlle':13,' Col':14,' Capt':15,' the Countess':16,' Jonkheer':17,' Dona':18 })
data['Embarked'] = data['Embarked'].map({'C': 1, 'Q': 2, 'S': 3})
data['CabinLetter']=data['CabinLetter'].map({'A':1, 'B':2, 'C':3, 'D':4, 'E':5, 'F':6, 'G':7, 'T':8})
data['CabinNumber'] = data['CabinNumber'].map({'0': 0, 8.0:8, 1.0:1,4.0 :4, 6.0:6, 5.0:5, 2.0:2, 7.0:7, 3.0:3, ' ': 0, '2': 2, '3':3, '5':5, '1':1, '4':4,'8':8, '7':7,'6':6, '9':9})
data.head()

,PassengerId,Survived,Sex,Age,Title,Pclass,CabinNumber,CabinLetter,Embarked,Fare,...,Parch,Cabin,FamilySize,Singleton,SmallFamily,LargeFamily,TicketCategory_Common,TicketCategory_Commonest,TicketCategory_Frequent,TicketCategory_Rare
0,1,0.0,0,22,1,3,0,6,3,7.2500,...,0,NaN,2,0,1,0,0,0,0,1
1,2,1.0,1,38,2,1,8,3,1,71.2833,...,0,C85,2,0,1,0,0,0,1,0
2,3,1.0,1,26,3,3,0,6,3,7.9250,...,0,NaN,1,1,0,0,0,0,1,0
3,4,1.0,1,35,2,1,1,3,3,53.1000,...,0,C123,2,0,1,0,0,0,1,0
4,5,0.0,0,35,1,3,0,6,3,8.0500,...,0,NaN,1,1,0,0,0,1,0,0


In [222]:
data['CabinLetter'].unique()

array([6, 3, 5, 4, 7, 1, 2, 8])

In [223]:
data.drop(columns=["SibSp"], inplace= True)
data.drop(columns=["Parch"], inplace= True)
data.head()


,PassengerId,Survived,Sex,Age,Title,Pclass,CabinNumber,CabinLetter,Embarked,Fare,Cabin,FamilySize,Singleton,SmallFamily,LargeFamily,TicketCategory_Common,TicketCategory_Commonest,TicketCategory_Frequent,TicketCategory_Rare
0,1,0.0,0,22,1,3,0,6,3,7.2500,NaN,2,0,1,0,0,0,0,1
1,2,1.0,1,38,2,1,8,3,1,71.2833,C85,2,0,1,0,0,0,1,0
2,3,1.0,1,26,3,3,0,6,3,7.9250,NaN,1,1,0,0,0,0,1,0
3,4,1.0,1,35,2,1,1,3,3,53.1000,C123,2,0,1,0,0,0,1,0
4,5,0.0,0,35,1,3,0,6,3,8.0500,NaN,1,1,0,0,0,1,0,0


In [224]:
data.drop(columns=["Cabin"], inplace= True)

In [225]:
data.head()

,PassengerId,Survived,Sex,Age,Title,Pclass,CabinNumber,CabinLetter,Embarked,Fare,FamilySize,Singleton,SmallFamily,LargeFamily,TicketCategory_Common,TicketCategory_Commonest,TicketCategory_Frequent,TicketCategory_Rare
0,1,0.0,0,22,1,3,0,6,3,7.2500,2,0,1,0,0,0,0,1
1,2,1.0,1,38,2,1,8,3,1,71.2833,2,0,1,0,0,0,1,0
2,3,1.0,1,26,3,3,0,6,3,7.9250,1,1,0,0,0,0,1,0
3,4,1.0,1,35,2,1,1,3,3,53.1000,2,0,1,0,0,0,1,0
4,5,0.0,0,35,1,3,0,6,3,8.0500,1,1,0,0,0,1,0,0


In [226]:
#Define the training and test dataset :
data_train=data[data['Survived'].notnull()]
data_test=data[data['Survived'].isnull()]
data_test.columns = ['PassengerId','Survived','Sex','Age','Title','Pclass','CabinNumber','CabinLetter','Embarked','Fare','FamilySize','Singleton','SmallFamily','LargeFamily','TicketCategory_Common','TicketCategory_Commonest','TicketCategory_Frequent','TicketCategory_Rare']


In [227]:
Y_train = data_train['Survived']
Y_test = data_test['Survived']
X_train = data_train.loc[:,data_train.columns!='Survived']
X_test = data_test.loc[:,data_test.columns!= "Survived"]


In [228]:

from sklearn.model_selection import train_test_split
X_training,X_validation,Y_training,Y_validation = train_test_split(X_train,Y_train, test_size = 0.3, random_state = 21)


In [229]:
X_training.head(15)

,PassengerId,Sex,Age,Title,Pclass,CabinNumber,CabinLetter,Embarked,Fare,FamilySize,Singleton,SmallFamily,LargeFamily,TicketCategory_Common,TicketCategory_Commonest,TicketCategory_Frequent,TicketCategory_Rare
241,242,1,22,3,3,0,6,2,15.5000,2,0,1,0,0,1,0,0
734,735,0,23,1,2,0,4,3,13.0000,1,1,0,0,1,0,0,0
581,582,1,39,2,1,6,3,1,110.8833,3,0,1,0,0,0,1,0
456,457,0,65,1,1,3,5,3,26.5500,1,1,0,0,0,0,1,0
118,119,0,24,1,1,5,2,1,247.5208,2,0,1,0,0,0,1,0
621,622,0,42,1,1,1,4,3,52.5542,2,0,1,0,0,0,1,0
162,163,0,26,1,3,0,5,3,7.7750,1,1,0,0,0,1,0,0
46,47,0,25,1,3,0,6,2,15.5000,2,0,1,0,0,1,0,0
709,710,0,25,4,3,0,7,1,15.2458,3,0,1,0,1,0,0,0
647,648,0,56,14,1,2,1,1,35.5000,1,1,0,0,0,0,1,0


In [230]:
count_nan = len(X_validation) - X_validation.count()
print(count_nan)

PassengerId                 0
Sex                         0
Age                         0
Title                       0
Pclass                      0
CabinNumber                 0
CabinLetter                 0
Embarked                    0
Fare                        0
FamilySize                  0
Singleton                   0
SmallFamily                 0
LargeFamily                 0
TicketCategory_Common       0
TicketCategory_Commonest    0
TicketCategory_Frequent     0
TicketCategory_Rare         0
dtype: int64


In [231]:
from sklearn.svm import SVC
from sklearn import preprocessing
from sklearn.metrics import make_scorer, accuracy_score, f1_score
from sklearn.model_selection import GridSearchCV


# In[43]:


train_test = pd.concat([X_training, X_test, X_validation], ignore_index=True)
train_test_normalized = preprocessing.scale(train_test)
X_training_normalized = train_test_normalized[:len(X_training), :]
X_test_normalized = train_test_normalized[len(X_training):len(X_training) + len(X_test), :]
X_validation_normalized = train_test_normalized[len(X_training) + len(X_test):, :]




In [232]:
svm_params = {
    'C' : [0.1, 0.3, 0.8, 0.9, 1.0, 2.0],
    'kernel' : ['linear', 'poly', 'rbf', 'sigmoid'],
    'tol' : [1e-3, 1e-4],
    'degree' : [2, 3, 4, 5],
    'random_state' : [21]
}


# In[50]:


acc_scorer = make_scorer(accuracy_score)
svc = SVC()
svc_classifiers = GridSearchCV(svc, svm_params, scoring=acc_scorer, n_jobs=-1)
svc_classifiers = svc_classifiers.fit(X_training_normalized, Y_training)

svc_best = svc_classifiers.best_estimator_
svc_best = svc_best.fit(X_training_normalized, Y_training)


# In[51]:


svc_model = {
    'Name' : 'SVC', 
    'CVScore' : svc_classifiers.best_score_, 
    'CVStd' : svc_classifiers.cv_results_['std_test_score'][svc_classifiers.best_index_],
    'Result_train' : svc_best.predict(X_training_normalized),
    'Result_test' : svc_best.predict(X_test_normalized),
    'Model' : svc_best
}


# In[73]:


best_idx = svc_classifiers.best_index_
print('Best model - avg:', 
      svc_model['CVScore'], 
      '+/-', 
      svc_model['CVStd'])
print()
print(svc_classifiers.best_estimator_)

Best model - avg: 0.826645264847512 +/- 0.016331312552120822

SVC(C=0.8, cache_size=200, class_weight=None, coef0=0.0,
  decision_function_shape='ovr', degree=2, gamma='auto', kernel='rbf',
  max_iter=-1, probability=False, random_state=21, shrinking=True,
  tol=0.001, verbose=False)


In [233]:
Y_pred = svc_best.predict(X_test_normalized)


# In[54]:


FINAL = pd.DataFrame({
        "PassengerId": data_test["PassengerId"].astype(int),
        "Survived": Y_pred.astype(int)
    })


# In[ ]:


FINAL.to_csv('predictionSVM41cc.csv',sep=',',index=False)



In [234]:
# Random Forest 
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import StratifiedKFold


# In[56]:


rf = RandomForestClassifier()


# In[57]:


run_gs = False

if run_gs:
    parameter_grid = {
                 'max_depth' : [4, 6, 8],
                 'n_estimators': [50, 10],
                 'max_features': ['sqrt', 'auto', 'log2'],
                 'min_samples_split': [2, 3, 10],
                 'min_samples_leaf': [1, 3, 10],
                 'bootstrap': [True, False],
                 }
    forest = RandomForestClassifier()
    cross_validation = StratifiedKFold(n_splits=5)

    grid_search = GridSearchCV(forest,
                               scoring='accuracy',
                               param_grid=parameter_grid,
                               cv=cross_validation,
                               verbose=1
                              )

    grid_search.fit(X_train, Y_train)
    model = grid_search
    parameters = grid_search.best_params_

    print('Best score: {}'.format(grid_search.best_score_))
    print('Best parameters: {}'.format(grid_search.best_params_))
    
else: 
    parameters = {'bootstrap': False, 'min_samples_leaf': 3, 'n_estimators': 50, 
                  'min_samples_split': 10, 'max_features': 'sqrt', 'max_depth': 6}
    
    model = RandomForestClassifier(**parameters)
    model.fit(X_train, Y_train)


# In[58]:


output = model.predict(X_test).astype(int)



# In[ ]:


FINAL2 = pd.DataFrame({
        "PassengerId": data_test["PassengerId"].astype(int),
        "Survived": output.astype(int)
    })
FINAL2.to_csv('predictionRFtest41cc.csv',sep=',',index=False)



In [235]:
from sklearn.neighbors import KNeighborsClassifier


# In[74]:


param_grid = {"n_neighbors":np.arange(1,50)}
knn =  KNeighborsClassifier()
knn_cv = GridSearchCV(knn,param_grid, cv=10)
knn_cv.fit(X_training,Y_training)
knn_cv.best_params_
knn_cv.best_score_


# In[75]:


result = knn_cv.predict(X_test).astype(int)


# In[80]:


FINAL3 = pd.DataFrame({
        "PassengerId": data_test["PassengerId"].astype(int),
        "Survived": result.astype(int)
    })
FINAL3.to_csv('predictionKNN41cc.csv',sep=',',index=False)

